# Sakata method

# Feature Engineering

In [66]:
%matplotlib inline

import xgboost

In [67]:
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error,r2_score
import time
from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings("ignore")

# OS
import glob, re
from datetime import datetime
import pickle

# data science tool
import numpy as np
import pandas as pd
import datetime as dt


# machine learning
from sklearn import *
from xgboost import XGBRegressor
import xgboost as xgb
import lightgbm as lgb

# データの読み込み
# 事前にcalendar_dateをvisit_dataに変更しています。airとhpgで同じことですが、別名で使用されているようです。
data = {
    'tra': pd.read_csv('../../../mltestdata/05_recruit/air_visit_data.csv'),
    'as': pd.read_csv('../../../mltestdata/05_recruit/air_store_info.csv'),
    'hs': pd.read_csv('../../../mltestdata/05_recruit/hpg_store_info.csv'),
    'ar': pd.read_csv('../../../mltestdata/05_recruit/air_reserve.csv'),
    'hr': pd.read_csv('../../../mltestdata/05_recruit/hpg_reserve.csv'),
    'id': pd.read_csv('../../../mltestdata/05_recruit/store_id_relation.csv'),
    'tes': pd.read_csv('../../../mltestdata/05_recruit/sample_submission.csv'),
    'hol': pd.read_csv('../../../mltestdata/05_recruit/date_info.csv').rename(columns={'calendar_date':'visit_date'})
}

# それぞれのデータをマージするために、まずは、relation用のものをマージします
data['hr'] = pd.merge(data['hr'], data['id'], how = 'inner', on = ['hpg_store_id'])

for df in ['ar', 'hr']:
    data[df]['visit_datetime'] = pd.to_datetime(data[df]['visit_datetime'])
    data[df]['visit_datetime'] = data[df]['visit_datetime'].dt.date
    data[df]['reserve_datetime'] = pd.to_datetime(data[df]['reserve_datetime'])
    data[df]['reserve_datetime'] = data[df]['reserve_datetime'].dt.date
    data[df]['reserve_datetime_diff'] = data[df].apply(lambda r:(r['visit_datetime']- r['reserve_datetime']).days, axis = 1)
    tmp1 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].sum().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs1', 'reserve_visitors':'rv1'})
    tmp2 = data[df].groupby(['air_store_id', 'visit_datetime'], as_index =False)[['reserve_datetime_diff', 'reserve_visitors']].mean().rename(columns = {'visit_datetime':'visit_date', 'reserve_datetime_diff':'rs2', 'reserve_visitors':'rv2'})
    data[df] = pd.merge(tmp1, tmp2, how = 'inner', on = ['air_store_id', 'visit_date'])

data['tra']['visit_date'] = pd.to_datetime(data['tra']['visit_date'])
data['tra']['dow'] = data['tra']['visit_date'].dt.dayofweek
data['tra']['year'] = data['tra']['visit_date'].dt.year
data['tra']['month'] = data['tra']['visit_date'].dt.month
data['tra']['visit_date'] = data['tra']['visit_date'].dt.date

data['tes']['visit_date'] = data['tes']['id'].map(lambda x: str(x).split('_')[2])
data['tes']['air_store_id'] = data['tes']['id'].map(lambda x: '_'.join(x.split('_')[:2]))
data['tes']['visit_date'] = pd.to_datetime(data['tes']['visit_date'])
data['tes']['dow'] = data['tes']['visit_date'].dt.dayofweek
data['tes']['year'] = data['tes']['visit_date'].dt.year
data['tes']['month'] = data['tes']['visit_date'].dt.month
data['tes']['visit_date'] = data['tes']['visit_date'].dt.date

unique_stores = data['tes']['air_store_id'].unique()

stores = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'dow':[i]*len(unique_stores)}) for i in range(7)], axis =0, ignore_index = True).reset_index(drop = True) 

#曜日だけでなく、月も追加
stores_m = pd.concat([pd.DataFrame({'air_store_id':unique_stores, 'month':[i]*len(unique_stores)}) for i in range(1,13)], axis =0, ignore_index = True).reset_index(drop = True)
stores = pd.merge(stores_m, stores,on=('air_store_id'), how='left')

tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].min().rename(columns = {'visitors':'min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].median().rename(columns = {'visitors':'median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].max().rename(columns = {'visitors':'max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])
tmp = data['tra'].groupby(['air_store_id', 'dow'], as_index = False)['visitors'].count().rename(columns = {'visitors':'count_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'dow'])

#曜日だけでなく、ID×月も追加
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].min().rename(columns = {'visitors':'m_min_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'m_mean_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].median().rename(columns = {'visitors':'m_median_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].max().rename(columns = {'visitors':'m_max_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])
tmp = data['tra'].groupby(['air_store_id', 'month'], as_index = False)['visitors'].count().rename(columns = {'visitors':'m_count_visitors'})
stores = pd.merge(stores, tmp, how ='left', on = ['air_store_id', 'month'])

stores = pd.merge(stores, data['as'], how= "left", on = ['air_store_id'])

stores['air_genre_name'] = stores['air_genre_name'].map(lambda x: str(str(x).replace('/', ' ')))
stores['air_area_name'] = stores['air_area_name'].map(lambda x: str(str(x).replace('-', ' ')))

lbl_gn = preprocessing.LabelEncoder()
lbl_an = preprocessing.LabelEncoder()

for i in range(10):
    stores['air_genre_name' + str(i)] = lbl_gn.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
    stores['air_area_name' + str(i)] = lbl_an.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
stores['air_genre_name'] = lbl_gn.fit_transform(stores['air_genre_name'])
stores['air_area_name'] = lbl_an.fit_transform(stores['air_area_name'])


#lbl = preprocessing.LabelEncoder()
#for i in range(10):
#    stores['air_genre_name' + str(i)] = lbl.fit_transform(stores['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
#    stores['air_area_name' + str(i)] = lbl.fit_transform(stores['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else ' '))
#stores['air_genre_name'] = lbl.fit_transform(stores['air_genre_name'])
#stores['air_area_name'] = lbl.fit_transform(stores['air_area_name'])


#土日フラグ(day_of_week_1)と、休日前(holi_2)フラグを追加
lbl = preprocessing.LabelEncoder()

data['hol']['day_of_week_1']= data['hol']['day_of_week'].replace(['Saturday', 'Sunday','Monday','Tuesday','Wednesday','Thursday','Friday'],['1', '1','0','0','0','0','0']).astype('int')
data['hol']['holi_2'] = data['hol'][['holiday_flg', 'day_of_week_1']].sum(axis = 1)
data['hol']['holi_2'] = data['hol']['holi_2'].apply( lambda x: 0 if x < 1 else 1 )
data['hol']['holi_2'] = data['hol']['holi_2'].shift(-1)
data['hol']['holi_2'] = data['hol']['holi_2'].fillna(1)
data['hol']['holi_2'] = data['hol']['holi_2'].astype('int')

data['hol']['visit_date'] = pd.to_datetime(data['hol']['visit_date'])
data['hol']['day_of_week'] = lbl.fit_transform(data['hol']['day_of_week'])
data['hol']['visit_date'] = data['hol']['visit_date'].dt.date
train = pd.merge(data['tra'], data['hol'], how ='left', on = ['visit_date'])
test = pd.merge(data['tes'], data['hol'], how ='left', on = ['visit_date'])

#曜日と月でmerge
train = pd.merge(train, stores, how ='left', on = ['air_store_id', 'dow','month'])
test = pd.merge(test, stores, how ='left', on = ['air_store_id', 'dow','month'])

#ID×休日前でのvisitorsの平均、中央値等を追加
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].min().rename(columns = {'visitors':'h_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'h_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].median().rename(columns = {'visitors':'h_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].max().rename(columns = {'visitors':'h_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])
tmp = train.groupby(['air_store_id','holi_2'], as_index = False)['visitors'].count().rename(columns = {'visitors':'h_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','holi_2'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','holi_2'])

for df in ['ar','hr']:
    train = pd.merge(train, data[df], how='left', on=['air_store_id','visit_date']) 
    test = pd.merge(test, data[df], how='left', on=['air_store_id','visit_date'])

train['id'] = train.apply(lambda r: '_'.join([str(r['air_store_id']), str(r['visit_date'])]), axis=1) 

train['total_reserve_sum'] = train['rv1_x'] + train['rv1_y']
train['total_reserve_mean'] = (train['rv2_x'] + train['rv2_y'])/2
train['total_reserve_dt_diff_mean'] = (train['rs2_x'] + train['rs2_y'])/2

test['total_reserve_sum'] = test['rv1_x'] + test['rv1_y']
test['total_reserve_mean'] = (test['rv2_x'] + test['rv2_y'])/2
test['total_reserve_dt_diff_mean'] = (test['rs2_x'] + test['rs2_y'])/2

train['date_int'] = train['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
test['date_int'] = test['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
train['var_max_lat'] = train['latitude'].max() - train['latitude']
train['var_max_long'] = train['longitude'].max() - train['longitude']
test['var_max_lat'] = test['latitude'].max() - test['latitude']
test['var_max_long'] = test['longitude'].max() - test['longitude']
train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

lbl = preprocessing.LabelEncoder()
train['air_store_id2'] = lbl.fit_transform(train['air_store_id'])
test['air_store_id2']= lbl.fit_transform(test['air_store_id'])

train['lon_plus_lat'] = train['longitude'] + train['latitude'] 
test['lon_plus_lat'] = test['longitude'] + test['latitude']

col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date', 'visitors']]
train = train.fillna(-1)
test = test.fillna(-1)

ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat([train, test]) 

#指数移動平均の追加。これはなくても良いかも
#https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/discussion/46179#266344
#def calc_shifted_ewm(series, alpha, adjust=True):
#    return series.shift().ewm(alpha=alpha, adjust=adjust).mean()

#train['ewm'] = train.groupby(['air_store_id', 'dow']).apply(lambda g: calc_shifted_ewm(g['visitors'], 0.1)).sort_index(level=['air_store_id']).values

#以下、気象データの追加
df_air_store_weather_station = pd.read_csv('../../../mltestdata/05_recruit/air_store_info_with_nearest_active_station.csv')

cols = ['air_store_id', 'station_id', 'station_latitude', 'station_longitude', 'station_vincenty', 'station_great_circle']
all_data = pd.merge(all_data, df_air_store_weather_station[cols], on='air_store_id', how='left')

combine = all_data
filenames = []
df_weather = None
for station_id in combine['station_id'].unique():
    fn = f"../../../mltestdata/05_recruit/1-1-16_5-31-17_Weather/{station_id}.csv"
    if not fn in filenames:
        df = pd.read_csv(fn)
        df['station_id'] = station_id
        if df_weather is None:
            df_weather = df
        else:
            df_weather = pd.concat([df_weather, df])
        del df

        filenames.append(fn)
    else:
        continue

#欠損値を平均で穴埋め（median, ffillで試すも特に差は出なかった）
df_weather = df_weather.fillna(df_weather.mean())

df_weather = df_weather.rename(columns={'calendar_date': 'visit_date'})

df_weather['visit_date'] = pd.to_datetime(df_weather['visit_date'])
df_weather['visit_date'] = df_weather['visit_date'].dt.date

#なんとなく対数化
df_weather['precipitation'] = np.log1p(df_weather['precipitation'])

#使いそうなデータだけ結合（その他の気象データは試していません。特に意味はなし）
cols = ['station_id', 
    'visit_date', 
    'precipitation', 
    'hours_sunlight',
    'avg_temperature',
    'high_temperature',
    'low_temperature']

combine = pd.merge(combine, df_weather[cols], on=['station_id', 'visit_date'], how='left')

#降水量をカテゴリ化
def simplify_pre(df):
    df.precipitation = df.precipitation.fillna(0)
    bins = ( -1, 0.01, 2,  5)
    group_names = ['1', '2', '3']
    categories = pd.cut(df.precipitation, bins, labels=group_names)
    df.precipitation = categories
    return df

combine = simplify_pre(combine) 
all_data = combine 

#不要そうなデータを削除
drop_col =['station_id', 'station_latitude','station_longitude','station_vincenty', 'station_great_circle','hours_sunlight','high_temperature','low_temperature']
all_data = all_data.drop(drop_col, axis = 1)

train = all_data[:ntrain]
test = all_data[ntrain:]

#ID×降水量で平均、中央値等を追加
tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].min().rename(columns = {'visitors':'p_min_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].mean().rename(columns = {'visitors':'p_mean_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].median().rename(columns = {'visitors':'p_median_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation',])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].max().rename(columns = {'visitors':'p_max_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

tmp = train.groupby(['air_store_id','precipitation'], as_index = False)['visitors'].count().rename(columns = {'visitors':'p_count_visitors'})
train = pd.merge(train, tmp, how ='left', on = ['air_store_id','precipitation'])
test = pd.merge(test, tmp, how ='left', on = ['air_store_id','precipitation'])

#countをLabel Encoder化。なんとなく試してみたら結果が良かった。
lbl = preprocessing.LabelEncoder()
train['count_visitors'] = lbl.fit_transform(train['count_visitors']) 
test['count_visitors']= lbl.fit_transform(test['count_visitors'])
train['m_count_visitors'] = lbl.fit_transform(train['m_count_visitors'])
test['m_count_visitors']= lbl.fit_transform(test['m_count_visitors'])
train['h_count_visitors'] = lbl.fit_transform(train['h_count_visitors'])
test['h_count_visitors']= lbl.fit_transform(test['h_count_visitors'])
train['p_count_visitors'] = lbl.fit_transform(train['p_count_visitors'])
test['p_count_visitors']= lbl.fit_transform(test['p_count_visitors'])

# GW flag
combine = [train, test]
gw_list = ['2016-04-29','2016-04-30','2016-05-01','2016-05-02','2016-05-03','2016-05-04','2016-05-05','2017-04-29','2017-04-30','2017-05-01','2017-05-02','2017-05-03','2017-05-04','2017-05-05']
post_gw_list=['2016-05-06']
train['gw_flg'] = 0
train['post_gw_flg'] = 0
test['gw_flg'] = 0
test['post_gw_flg'] = 0
update_gw_list = [["0" for i in range(3)] for j in range(len(gw_list))]
update_post_gw_list = [["0" for i in range(3)] for j in range(len(post_gw_list))]

from datetime import date
for index, gw_date in enumerate(gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_gw_list[index][col_i]=int(temp_figure)
        
    #print("{}  {}  {}".format(update_list[index][0],update_list[index][1],update_list[index][2]))
    
for index, gw_date in enumerate(post_gw_list):
    temp_list = gw_date.split("-")
    for col_i, temp_figure in enumerate(temp_list):
        update_post_gw_list[index][col_i]=int(temp_figure)

for dataset in combine:
    for index in range(len(update_gw_list)):
        dataset.loc[dataset.visit_date == date(update_gw_list[index][0],update_gw_list[index][1],update_gw_list[index][2]), 'gw_flg'] = 1
        
for dataset in combine:
    for index in range(len(update_post_gw_list)):
        dataset.loc[dataset.visit_date == date(update_post_gw_list[index][0],update_post_gw_list[index][1],update_post_gw_list[index][2]), 'post_gw_flg'] = 1   

In [68]:
train['precipitation'] = train['precipitation'].values.astype(float)
test['precipitation'] = test['precipitation'].values.astype(float)

train = train.fillna(-1)
test = test.fillna(-1)

In [69]:
train.shape

(252108, 75)

In [70]:
test.shape

(32019, 75)

# Data Separation

Refered to this [super cool eda](https://www.kaggle.com/c/recruit-restaurant-visitor-forecasting/discussion/45048). I will categorize as below.
- Izakaya
- Cafe Sweets
- others

In [71]:
lbl_gn.classes_

array(['Asian', 'Bar Cocktail', 'Cafe Sweets', 'Creative cuisine',
       'Dining bar', 'International cuisine', 'Italian French', 'Izakaya',
       'Japanese food', 'Karaoke Party', 'Okonomiyaki Monja Teppanyaki',
       'Other', 'Western food', 'Yakiniku Korean food'], dtype=object)

In [72]:
test.air_genre_name.value_counts()

7.0     7605
2.0     6903
4.0     4173
6.0     3939
1.0     3081
8.0     2457
11.0    1053
13.0     897
12.0     624
10.0     546
3.0      507
5.0       78
9.0       78
0.0       78
Name: air_genre_name, dtype: int64

In [73]:
print("Izakaya: "+str(lbl_gn.transform(['Izakaya'])))
print("Cafe Sweets: "+str(lbl_gn.transform(['Cafe Sweets'])))
print("Italian French: "+str(lbl_gn.transform(['Italian French'])))
print("Dining bar: "+str(lbl_gn.transform(['Dining bar'])))

Izakaya: [7]
Cafe Sweets: [2]
Italian French: [6]
Dining bar: [4]


In [74]:
x_train_iza = train[train.air_genre_name == 7]
x_train_caf = train[train.air_genre_name == 2]
x_train_ita = train[(train.air_genre_name == 4) | (train.air_genre_name == 6)]
x_train_oth = train[(train.air_genre_name != 7) & (train.air_genre_name != 2) & (train.air_genre_name != 4) & (train.air_genre_name != 6)]

y_train_iza=x_train_iza.visitors
y_train_caf=x_train_caf.visitors
y_train_ita=x_train_ita.visitors
y_train_oth=x_train_oth.visitors

drop_cols=['visitors','air_store_id','visit_date','id']
x_train_iza=x_train_iza.drop(drop_cols, axis=1)
x_train_caf=x_train_caf.drop(drop_cols, axis=1)
x_train_ita=x_train_ita.drop(drop_cols, axis=1)
x_train_oth=x_train_oth.drop(drop_cols, axis=1)

x_test_iza = test[test.air_genre_name == 7]
x_test_caf = test[test.air_genre_name == 2]
x_test_ita = test[(test.air_genre_name == 4) | (test.air_genre_name == 6)]
x_test_oth = test[(test.air_genre_name != 7) & (test.air_genre_name != 2) & (test.air_genre_name != 4) & (test.air_genre_name != 6)]

x_test_iza=x_test_iza.drop(drop_cols, axis=1)
x_test_caf=x_test_caf.drop(drop_cols, axis=1)
x_test_ita=x_test_ita.drop(drop_cols, axis=1)
x_test_oth=x_test_oth.drop(drop_cols, axis=1)

In [75]:
lbl_an.inverse_transform(30)

'Hyōgo ken Nishinomiya shi Rokutanjichō'

In [76]:
lbl_gn.inverse_transform(4)

'Dining bar'

In [77]:
lbl_an.inverse_transform(82)

'Tōkyō to Taitō ku Higashiueno'

In [78]:
print("gn: "+str(test[175:176].air_genre_name))
print("an: "+str(test[175:176].air_area_name))

gn: 175    2.0
Name: air_genre_name, dtype: float64
an: 175    102.0
Name: air_area_name, dtype: float64


In [79]:
print("gn: "+str(test[225:226].air_genre_name))
print("an: "+str(test[225:226].air_area_name))

gn: 225    2.0
Name: air_genre_name, dtype: float64
an: 225    31.0
Name: air_area_name, dtype: float64


In [80]:
print("gn: "+str(test[575:576].air_genre_name))
print("an: "+str(test[575:576].air_area_name))

gn: 575    2.0
Name: air_genre_name, dtype: float64
an: 575    30.0
Name: air_area_name, dtype: float64


# Test with lgbm

### Utility function

In [81]:
#Define a evaluation function

def rmsle(preds, true):
    #rmsle = np.sqrt(mean_squared_error(np.float64(np.log1p(true)), np.float64(np.log1p(preds))))
    rmsle = np.sqrt(mean_squared_error(np.log1p(true), np.log1p(preds)))
    return float(rmsle)

#Define a evaluation matrix 
from sklearn.metrics.scorer import make_scorer
RMSLE = make_scorer(rmsle)

# Define a function for comparing predictions and true data.
def compare_result(preds, true):
    compare = pd.DataFrame({"test_id": true.index,
                           "real_cost": true,
                           "pred_cost": preds})
    compare = compare[["test_id", "real_cost", "pred_cost"]].reset_index(drop=True)
    
    compare["error_percent_(%)"] = np.abs(compare.real_cost - compare.pred_cost) / compare.real_cost * 100
    
    return compare

In [82]:
def cross_validate_xgb(params, x_train, y_train, x_test, kf,  verbose=True, verbose_eval=50, scoreonly=False):
    start_time=time.time()
    nround=[]
    # the prediction matrix need to contains 3 columns, one for the probability of each class
    #train_pred = np.zeros((x_train.shape[0],3))
    train_pred = np.zeros((x_train.shape[0]))
    test_pred = np.zeros((x_test.shape[0]))
    
    # self-defined eval metric
    # f(preds: array, train_data: Dataset) -> name: string, value: array, is_higher_better: bool
    # binary error
    def feval_rmsle(preds, train_data):
        preds = np.expm1(preds)
        true = np.expm1(train_data.get_label())
        #return 'rmsle', rmsle(true, preds), False

        return 'rmsle', rmsle(preds, true), False

    # use the k-fold object to enumerate indexes for each training and validation fold
    for i, (train_index, val_index) in enumerate(kf.split(x_train, y_train)):
        x_train_kf, x_val_kf = x_train.loc[train_index, :], x_train.loc[val_index, :]

        #y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
        y_train_kf, y_val_kf = np.log1p(y_train[train_index]), np.log1p(y_train[val_index])
        x_test_kf=x_test.copy()
        
        d_train = xgboost.DMatrix(x_train_kf, y_train_kf)
        d_val=xgboost.DMatrix(x_val_kf, y_val_kf)
        d_test = xgboost.DMatrix(x_test_kf)
        
        watchlist= [(d_train, "train"), (d_val, 'val')]
        bst = xgboost.train(params=params, 
                            dtrain=d_train, 
                            num_boost_round=8000, 
                            early_stopping_rounds=100,
                            evals=watchlist, 
                            verbose_eval=verbose_eval)        
        
#        y_val_kf_preds=bst.predict(d_val, ntree_limit=bst.best_ntree_limit)
        y_val_kf_preds=np.expm1(bst.predict(d_val, ntree_limit=bst.best_ntree_limit))
        nround.append(bst.best_ntree_limit)
        
        train_pred[val_index] += y_val_kf_preds
#        test_pred += np.expm1((bst.predict(x_test, ntree_limit=bst.best_ntree_limit)))
        test_pred += np.expm1(bst.predict(d_test, ntree_limit=bst.best_ntree_limit))
        
        
        #fold_cv = log_loss(y_val_kf.values, y_val_kf_preds)
        #fold_rmsle = rmsle(np.expm1(train_pred[val_index]),np.expm1(y_val_kf.values))
        fold_rmsle = rmsle(train_pred[val_index],np.expm1(y_val_kf.values))
        fold_cv = fold_rmsle
        
        if verbose:
            print('fold cv {} rmsle score is {:.6f}'.format(i, fold_cv))

    test_pred = test_pred / kf.n_splits
    #cv_score = log_loss(y_train, train_pred)
    #cv_score = rmsle(np.expm1(train_pred), y_train)
    cv_score = rmsle(train_pred, y_train)
    
    if verbose:
        print('cv rmsle score is {:.6f}'.format(cv_score))    
        end_time = time.time()
        print("it takes %.3f seconds to perform cross validation" % (end_time - start_time))
 
    if scoreonly:
        #return cv_score # for the purpose of bayesian optimisation, we only need to return the CV score
        return cv_score
    else:
        return (cv_score,train_pred,test_pred)

In [83]:
# only do 3 fold CV here so that we save some running time on Kaggle Kernel
kf=StratifiedKFold(n_splits=10, shuffle=True, random_state=2018)

In [84]:
#行数
pd.set_option("display.max_rows", 101)

### 1. Train for Izakaya

In [89]:
xgb_params = {
    "objective" : "reg:linear",
    "eval_metric" : "rmse",
    "nthread": 4,
    "seed" : 0,
    'silent': 1,

    "eta":0.02,  # default 0.3
    "max_depth" : 5, # default 6
    "subsample" : 0.8, # default 1
    "colsample_bytree" : 0.6, # default 1
    "gamma": 0.5
}

In [90]:
X_train, X_test, y_train, y_test = train_test_split(x_train_iza,y_train_iza,test_size=0.2,random_state=2018)

y_train_log, y_test_log = np.log1p(y_train), np.log1p(y_test)

d_train = xgboost.DMatrix(X_train, y_train_log)
d_val=xgboost.DMatrix(X_test, y_test_log)
d_test = xgboost.DMatrix(x_test_iza)

watchlist= [(d_train, "train"), (d_val, 'val')]
bst = xgboost.train(params=xgb_params, 
                    dtrain=d_train, 
                    num_boost_round=8000, 
                    early_stopping_rounds=100,
                    evals=watchlist, 
                    verbose_eval=50)

[0]	train-rmse:2.49281	val-rmse:2.49244
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:1.02485	val-rmse:1.02664
[100]	train-rmse:0.593777	val-rmse:0.599106
[150]	train-rmse:0.502984	val-rmse:0.51084
[200]	train-rmse:0.484967	val-rmse:0.494752
[250]	train-rmse:0.479075	val-rmse:0.49031
[300]	train-rmse:0.47594	val-rmse:0.488404
[350]	train-rmse:0.47359	val-rmse:0.486983
[400]	train-rmse:0.471986	val-rmse:0.486132
[450]	train-rmse:0.470334	val-rmse:0.485406
[500]	train-rmse:0.469209	val-rmse:0.484943
[550]	train-rmse:0.467943	val-rmse:0.484308
[600]	train-rmse:0.467065	val-rmse:0.483952
[650]	train-rmse:0.466117	val-rmse:0.483603
[700]	train-rmse:0.465404	val-rmse:0.483342
[750]	train-rmse:0.464634	val-rmse:0.483133
[800]	train-rmse:0.463568	val-rmse:0.482841
[850]	train-rmse:0.462793	val-rmse:0.48257
[900]	train-rmse:0.462129	val-rmse:0.482346
[950]	train-rmse:0.461261	val-rmse

In [91]:
y_pred_iza_log = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)
y_pred_iza = np.expm1(y_pred_iza_log)

x_test_iza["visitors"] = y_pred_iza

In [92]:
x_test_iza.shape

(7605, 72)

### 2. Train for Cafe

In [93]:
xgb_params = {
    "objective" : "reg:linear",
    "eval_metric" : "rmse",
    "nthread": 4,
    "seed" : 0,
    'silent': 1,

    "eta":0.02,  # default 0.3
    "max_depth" : 5, # default 6
    "subsample" : 0.8, # default 1
    "colsample_bytree" : 0.6, # default 1
    "gamma": 0.5
}

X_train, X_test, y_train, y_test = train_test_split(x_train_caf,y_train_caf,test_size=0.2,random_state=2018)

y_train_log, y_test_log = np.log1p(y_train), np.log1p(y_test)

d_train = xgboost.DMatrix(X_train, y_train_log)
d_val=xgboost.DMatrix(X_test, y_test_log)
d_test = xgboost.DMatrix(x_test_caf)

watchlist= [(d_train, "train"), (d_val, 'val')]
bst = xgboost.train(params=xgb_params, 
                    dtrain=d_train, 
                    num_boost_round=8000, 
                    early_stopping_rounds=100,
                    evals=watchlist, 
                    verbose_eval=50)

[0]	train-rmse:2.47053	val-rmse:2.4729
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.992627	val-rmse:0.994421
[100]	train-rmse:0.542249	val-rmse:0.547989
[150]	train-rmse:0.442016	val-rmse:0.451772
[200]	train-rmse:0.421323	val-rmse:0.433806
[250]	train-rmse:0.413937	val-rmse:0.428431
[300]	train-rmse:0.409562	val-rmse:0.425805
[350]	train-rmse:0.406607	val-rmse:0.424299
[400]	train-rmse:0.404147	val-rmse:0.423103
[450]	train-rmse:0.402066	val-rmse:0.422068
[500]	train-rmse:0.400276	val-rmse:0.421496
[550]	train-rmse:0.398358	val-rmse:0.420927
[600]	train-rmse:0.396683	val-rmse:0.420302
[650]	train-rmse:0.395214	val-rmse:0.419845
[700]	train-rmse:0.393672	val-rmse:0.419402
[750]	train-rmse:0.392237	val-rmse:0.419148
[800]	train-rmse:0.391072	val-rmse:0.418912
[850]	train-rmse:0.389834	val-rmse:0.418532
[900]	train-rmse:0.388673	val-rmse:0.418251
[950]	train-rmse:0.387483	va

In [94]:
y_pred_caf_log = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)
y_pred_caf = np.expm1(y_pred_caf_log)

x_test_caf["visitors"] = y_pred_caf

x_test_caf.shape

(6903, 72)

### 3. Train for Italian & French

In [95]:
xgb_params = {
    "objective" : "reg:linear",
    "eval_metric" : "rmse",
    "nthread": 4,
    "seed" : 0,
    'silent': 1,

    "eta":0.02,  # default 0.3
    "max_depth" : 5, # default 6
    "subsample" : 0.8, # default 1
    "colsample_bytree" : 0.6, # default 1
    "gamma": 0.5
}

X_train, X_test, y_train, y_test = train_test_split(x_train_ita,y_train_ita,test_size=0.2,random_state=2018)

y_train_log, y_test_log = np.log1p(y_train), np.log1p(y_test)

d_train = xgboost.DMatrix(X_train, y_train_log)
d_val=xgboost.DMatrix(X_test, y_test_log)
d_test = xgboost.DMatrix(x_test_ita)

watchlist= [(d_train, "train"), (d_val, 'val')]
bst = xgboost.train(params=xgb_params, 
                    dtrain=d_train, 
                    num_boost_round=8000, 
                    early_stopping_rounds=100,
                    evals=watchlist, 
                    verbose_eval=50)

[0]	train-rmse:2.36569	val-rmse:2.3682
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.993979	val-rmse:0.98983
[100]	train-rmse:0.603653	val-rmse:0.596722
[150]	train-rmse:0.523668	val-rmse:0.51776
[200]	train-rmse:0.506887	val-rmse:0.502886
[250]	train-rmse:0.501048	val-rmse:0.498777
[300]	train-rmse:0.497348	val-rmse:0.496777
[350]	train-rmse:0.494332	val-rmse:0.495456
[400]	train-rmse:0.491726	val-rmse:0.494505
[450]	train-rmse:0.489395	val-rmse:0.493702
[500]	train-rmse:0.487458	val-rmse:0.493179
[550]	train-rmse:0.485672	val-rmse:0.492664
[600]	train-rmse:0.48399	val-rmse:0.492245
[650]	train-rmse:0.482571	val-rmse:0.491815
[700]	train-rmse:0.481295	val-rmse:0.491583
[750]	train-rmse:0.480189	val-rmse:0.491392
[800]	train-rmse:0.479032	val-rmse:0.491202
[850]	train-rmse:0.478174	val-rmse:0.491134
[900]	train-rmse:0.477388	val-rmse:0.490915
[950]	train-rmse:0.476701	val-r

In [96]:
y_pred_ita_log = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)
y_pred_ita = np.expm1(y_pred_ita_log)

x_test_ita["visitors"] = y_pred_ita

x_test_ita.shape

(8112, 72)

### 4. Train for Others

In [97]:
xgb_params = {
    "objective" : "reg:linear",
    "eval_metric" : "rmse",
    "nthread": 4,
    "seed" : 0,
    'silent': 1,

    "eta":0.02,  # default 0.3
    "max_depth" : 5, # default 6
    "subsample" : 0.8, # default 1
    "colsample_bytree" : 0.6, # default 1
    "gamma": 0.5
}

X_train, X_test, y_train, y_test = train_test_split(x_train_oth,y_train_oth,test_size=0.2,random_state=2018)

y_train_log, y_test_log = np.log1p(y_train), np.log1p(y_test)

d_train = xgboost.DMatrix(X_train, y_train_log)
d_val=xgboost.DMatrix(X_test, y_test_log)
d_test = xgboost.DMatrix(x_test_oth)

watchlist= [(d_train, "train"), (d_val, 'val')]
bst = xgboost.train(params=xgb_params, 
                    dtrain=d_train, 
                    num_boost_round=8000, 
                    early_stopping_rounds=100,
                    evals=watchlist, 
                    verbose_eval=50)

[0]	train-rmse:2.27565	val-rmse:2.27948
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[50]	train-rmse:0.959318	val-rmse:0.960167
[100]	train-rmse:0.587249	val-rmse:0.58569
[150]	train-rmse:0.512069	val-rmse:0.51056
[200]	train-rmse:0.496622	val-rmse:0.496297
[250]	train-rmse:0.491056	val-rmse:0.492069
[300]	train-rmse:0.487598	val-rmse:0.490008
[350]	train-rmse:0.484632	val-rmse:0.488419
[400]	train-rmse:0.482148	val-rmse:0.487237
[450]	train-rmse:0.480129	val-rmse:0.486316
[500]	train-rmse:0.478165	val-rmse:0.485526
[550]	train-rmse:0.476502	val-rmse:0.48489
[600]	train-rmse:0.474814	val-rmse:0.484287
[650]	train-rmse:0.473332	val-rmse:0.483867
[700]	train-rmse:0.471843	val-rmse:0.483454
[750]	train-rmse:0.470421	val-rmse:0.483026
[800]	train-rmse:0.469004	val-rmse:0.48267
[850]	train-rmse:0.467604	val-rmse:0.482332
[900]	train-rmse:0.466241	val-rmse:0.481905
[950]	train-rmse:0.464891	val-r

In [99]:
y_pred_oth_log = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)
y_pred_oth = np.expm1(y_pred_oth_log)

x_test_oth["visitors"] = y_pred_oth

x_test_oth.shape

(9399, 72)

# Merge

In [100]:
print((0.478342+0.490264+0.479244)/3)

0.48261666666666664


In [101]:
test_pred = pd.concat([x_test_iza,x_test_caf,x_test_ita,x_test_oth])

In [102]:
test_pred.shape

(32019, 72)

In [103]:
test_pred=test_pred.sort_index()

# Submission

In [104]:
print('Start...')
test['visitors'] = test_pred.visitors
sub = test[['id','visitors']].copy()
sub.to_csv('submission_rs_recruit_v20_xgb_by_genre_01.csv', index=False)
print('Finished.')
# xgb
# fe
# sakata method
# LB: ---

Start...
Finished.


In [105]:
sub1 = sub.copy()

In [106]:
# from hklee
# https://www.kaggle.com/zeemeen/weighted-mean-comparisons-lb-0-497-1st/code
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn)for fn in glob.glob('../../../mltestdata/05_recruit/*.csv')}

for k, v in dfs.items(): locals()[k] = v

wkend_holidays = date_info.apply(
    (lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
date_info.loc[wkend_holidays, 'holiday_flg'] = 0
date_info['weight'] = ((date_info.index + 1) / len(date_info)) ** 5  

visit_data = air_visit_data.merge(date_info, left_on='visit_date', right_on='calendar_date', how='left')
visit_data.drop('calendar_date', axis=1, inplace=True)
visit_data['visitors'] = visit_data.visitors.map(pd.np.log1p)

wmean = lambda x:( (x.weight * x.visitors).sum() / x.weight.sum() )
visitors = visit_data.groupby(['air_store_id', 'day_of_week', 'holiday_flg']).apply(wmean).reset_index()
visitors.rename(columns={0:'visitors'}, inplace=True) # cumbersome, should be better ways.

sample_submission['air_store_id'] = sample_submission.id.map(lambda x: '_'.join(x.split('_')[:-1]))
sample_submission['calendar_date'] = sample_submission.id.map(lambda x: x.split('_')[2])
sample_submission.drop('visitors', axis=1, inplace=True)
sample_submission = sample_submission.merge(date_info, on='calendar_date', how='left')
sample_submission = sample_submission.merge(visitors, on=[
    'air_store_id', 'day_of_week', 'holiday_flg'], how='left')

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[visitors.holiday_flg==0], on=('air_store_id', 'day_of_week'), 
    how='left')['visitors_y'].values

missings = sample_submission.visitors.isnull()
sample_submission.loc[missings, 'visitors'] = sample_submission[missings].merge(
    visitors[['air_store_id', 'visitors']].groupby('air_store_id').mean().reset_index(), 
    on='air_store_id', how='left')['visitors_y'].values

sample_submission['visitors'] = sample_submission.visitors.map(pd.np.expm1)
sub2 = sample_submission[['id', 'visitors']].copy()
sub_merge = pd.merge(sub1, sub2, on='id', how='inner')

sub_merge['visitors'] = (sub_merge['visitors_x'] + sub_merge['visitors_y']* 1.1)/2
sub_merge[['id', 'visitors']].to_csv('submission_rs_recruit_v20_xgb_by_genre_02.csv', index=False)
# xgb
# fe
# sakata method
# weight
# LB: 0.481